# Computing PI - Performance Comparison

Ce notebook compare différentes méthodes pour calculer π avec deux algorithmes :
1. **Méthode par intégrale** : Approximation par la méthode des rectangles
2. **Méthode Metropolis** : Simulation Monte Carlo (points aléatoires dans un cercle)

Nous testerons avec Python natif, générateurs, lambda, Numpy, Cython, Numba, Pandas et Dask.

In [ ]:
# Imports nécessaires
import time
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from functools import wraps

# Configuration pour les tests
NUM_TRIALS = [1_000_000, 100_000_000]  # Augmenter progressivement selon les performances
# NUM_TRIALS = [1_000_000, 100_000_000, 10_000_000_000, 1_000_000_000_000]  # Version complète

# Dictionnaire pour stocker les résultats
results = {}

## 3.1 Native Python Implementation

### Question 3.1 : Implémentation des deux algorithmes

In [ ]:
def compute_pi_integral_python(num_trials):
    """
    Calcul de PI par la méthode de l'intégrale.
    Approxime l'intégrale de 4/(1+x²) de 0 à 1.
    """
    step = 1.0 / num_trials
    sum_val = 0.0
    
    for j in range(num_trials):
        x = (j - 0.5) * step
        sum_val += 4.0 / (1.0 + x * x)
    
    return sum_val * step


def compute_pi_metropolis_python(num_trials, seed=42):
    """
    Calcul de PI par la méthode de Metropolis (Monte Carlo).
    Simule des points aléatoires dans un carré et compte ceux dans le cercle.
    """
    import random
    random.seed(seed)
    counter = 0
    
    for j in range(num_trials):
        x_val = random.random()
        y_val = random.random()
        radius = x_val ** 2 + y_val ** 2
        
        if radius < 1:
            counter += 1
    
    return 4.0 * counter / num_trials


# Test rapide
print("Test avec 1,000,000 itérations:")
print(f"PI (intégrale) = {compute_pi_integral_python(1_000_000)}")
print(f"PI (metropolis) = {compute_pi_metropolis_python(1_000_000)}")
print(f"Valeur réelle π = {np.pi}")

### Questions 3.2-3.4 : Évaluation des performances et visualisation

In [ ]:
# Benchmark des implémentations Python natives
results['python_native'] = {'integral': [], 'metropolis': []}

for num_trial in NUM_TRIALS:
    print(f"\nTesting with {num_trial:,} iterations...")
    
    # Test intégrale
    start = time.time()
    pi_val = compute_pi_integral_python(num_trial)
    elapsed = time.time() - start
    results['python_native']['integral'].append(elapsed)
    print(f"  Integral: π = {pi_val:.6f}, Time = {elapsed:.4f}s")
    
    # Test Metropolis
    start = time.time()
    pi_val = compute_pi_metropolis_python(num_trial)
    elapsed = time.time() - start
    results['python_native']['metropolis'].append(elapsed)
    print(f"  Metropolis: π = {pi_val:.6f}, Time = {elapsed:.4f}s")

In [ ]:
# Visualisation des performances Python natif
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(NUM_TRIALS, results['python_native']['integral'], 'o-', label='Integral', linewidth=2)
ax.plot(NUM_TRIALS, results['python_native']['metropolis'], 's-', label='Metropolis', linewidth=2)
ax.set_xlabel('Number of trials', fontsize=12)
ax.set_ylabel('Time (seconds)', fontsize=12)
ax.set_title('Python Native Implementation Performance', fontsize=14)
ax.set_xscale('log')
ax.set_yscale('log')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

**Explication des performances (Python natif)** :
- La méthode par **intégrale** est plus rapide car elle effectue des calculs mathématiques simples
- La méthode **Metropolis** est plus lente à cause des appels répétés à `random.random()` et des comparaisons conditionnelles
- Les deux ont une complexité O(n) mais avec des constantes différentes

## 3.2 Generator and Lambda

### Questions 3.5-3.9 : Implémentations avec générateurs et lambda

In [ ]:
# Version générateur
def compute_pi_integral_generator(num_trials):
    """Version générateur de la méthode intégrale"""
    step = 1.0 / num_trials
    def value_generator():
        for j in range(num_trials):
            x = (j - 0.5) * step
            yield 4.0 / (1.0 + x * x)
    return sum(value_generator()) * step


def compute_pi_metropolis_generator(num_trials, seed=42):
    """Version générateur de la méthode Metropolis"""
    import random
    random.seed(seed)
    def point_generator():
        for _ in range(num_trials):
            x_val = random.random()
            y_val = random.random()
            yield 1 if (x_val ** 2 + y_val ** 2) < 1 else 0
    return 4.0 * sum(point_generator()) / num_trials


# Versions lambda
compute_pi_integral_lambda = lambda n: sum((lambda s: (4.0 / (1.0 + ((j - 0.5) * s) ** 2) for j in range(n)))(1.0 / n)) * (1.0 / n)

compute_pi_metropolis_lambda = lambda n, seed=42: (lambda r: (r.seed(seed), 4.0 * sum(1 for _ in range(n) if r.random() ** 2 + r.random() ** 2 < 1) / n)[1])(__import__('random'))


# Test
print("Test avec 1,000,000 itérations:")
print(f"PI (intégrale generator) = {compute_pi_integral_generator(1_000_000)}")
print(f"PI (metropolis generator) = {compute_pi_metropolis_generator(1_000_000)}")
print(f"PI (intégrale lambda) = {compute_pi_integral_lambda(1_000_000)}")
print(f"PI (metropolis lambda) = {compute_pi_metropolis_lambda(1_000_000)}")

## 3.3 Numpy

### Questions 3.10-3.14 : Implémentations avec Numpy et vectorisation

In [ ]:
def compute_pi_integral_numpy(num_trials):
    """Version Numpy vectorisée - méthode intégrale"""
    step = 1.0 / num_trials
    j = np.arange(num_trials)
    x = (j - 0.5) * step
    sum_val = np.sum(4.0 / (1.0 + x * x))
    return sum_val * step


def compute_pi_metropolis_numpy(num_trials, seed=42):
    """Version Numpy vectorisée - méthode Metropolis"""
    np.random.seed(seed)
    x_vals = np.random.random(num_trials)
    y_vals = np.random.random(num_trials)
    radius = x_vals ** 2 + y_vals ** 2
    counter = np.sum(radius < 1)
    return 4.0 * counter / num_trials


# Version avec np.vectorize (généralement pas plus rapide)
@np.vectorize
def integral_func(j, step):
    x = (j - 0.5) * step
    return 4.0 / (1.0 + x * x)

def compute_pi_integral_numpy_vectorize(num_trials):
    """Version avec np.vectorize"""
    step = 1.0 / num_trials
    j = np.arange(num_trials)
    return np.sum(integral_func(j, step)) * step


# Test
print("Test avec 1,000,000 itérations:")
print(f"PI (intégrale numpy) = {compute_pi_integral_numpy(1_000_000)}")
print(f"PI (metropolis numpy) = {compute_pi_metropolis_numpy(1_000_000)}")
print(f"PI (intégrale numpy vectorize) = {compute_pi_integral_numpy_vectorize(1_000_000)}")

## 3.5 Numba

### Questions 3.20-3.25 : Implémentations avec Numba et parallélisme

In [ ]:
import numba
from numba import njit, prange, vectorize

# Version Numba simple
@njit
def compute_pi_integral_numba(num_trials):
    """Version Numba JIT - méthode intégrale"""
    step = 1.0 / num_trials
    sum_val = 0.0
    
    for j in range(num_trials):
        x = (j - 0.5) * step
        sum_val += 4.0 / (1.0 + x * x)
    
    return sum_val * step


@njit
def compute_pi_metropolis_numba(num_trials, seed=42):
    """Version Numba JIT - méthode Metropolis"""
    np.random.seed(seed)
    counter = 0
    
    for j in range(num_trials):
        x_val = np.random.random()
        y_val = np.random.random()
        radius = x_val ** 2 + y_val ** 2
        
        if radius < 1:
            counter += 1
    
    return 4.0 * counter / num_trials


# Version Numba parallèle
@njit(parallel=True)
def compute_pi_integral_numba_parallel(num_trials):
    """Version Numba parallèle - méthode intégrale"""
    step = 1.0 / num_trials
    sum_val = 0.0
    
    for j in prange(num_trials):
        x = (j - 0.5) * step
        sum_val += 4.0 / (1.0 + x * x)
    
    return sum_val * step


@njit(parallel=True)
def compute_pi_metropolis_numba_parallel(num_trials, seed=42):
    """Version Numba parallèle - méthode Metropolis"""
    np.random.seed(seed)
    x_vals = np.random.random(num_trials)
    y_vals = np.random.random(num_trials)
    counter = 0
    
    for j in prange(num_trials):
        radius = x_vals[j] ** 2 + y_vals[j] ** 2
        if radius < 1:
            counter += 1
    
    return 4.0 * counter / num_trials


# Version vectorize
@vectorize(['float64(float64, float64)'], target='parallel')
def integral_vectorize(j, step):
    x = (j - 0.5) * step
    return 4.0 / (1.0 + x * x)

def compute_pi_integral_numba_vectorize(num_trials):
    """Version Numba vectorize"""
    step = 1.0 / num_trials
    j = np.arange(num_trials, dtype=np.float64)
    return np.sum(integral_vectorize(j, step)) * step


# Test (warm-up pour compilation JIT)
print("Warm-up (compilation JIT)...")
_ = compute_pi_integral_numba(1000)
_ = compute_pi_metropolis_numba(1000)
_ = compute_pi_integral_numba_parallel(1000)
_ = compute_pi_metropolis_numba_parallel(1000)

print("\nTest avec 1,000,000 itérations:")
print(f"PI (intégrale numba) = {compute_pi_integral_numba(1_000_000)}")
print(f"PI (metropolis numba) = {compute_pi_metropolis_numba(1_000_000)}")
print(f"PI (intégrale numba parallel) = {compute_pi_integral_numba_parallel(1_000_000)}")
print(f"PI (metropolis numba parallel) = {compute_pi_metropolis_numba_parallel(1_000_000)}")

## Benchmark complet et comparaison

Maintenant, comparons toutes les implémentations :

In [ ]:
# Dictionnaire des fonctions à tester
implementations = {
    'Python Native': (compute_pi_integral_python, compute_pi_metropolis_python),
    'Numpy': (compute_pi_integral_numpy, compute_pi_metropolis_numpy),
    'Numba': (compute_pi_integral_numba, compute_pi_metropolis_numba),
    'Numba Parallel': (compute_pi_integral_numba_parallel, compute_pi_metropolis_numba_parallel),
}

# Benchmark complet
benchmark_results = {name: {'integral': [], 'metropolis': []} for name in implementations}

for num_trial in NUM_TRIALS:
    print(f"\n{'='*60}")
    print(f"Testing with {num_trial:,} iterations")
    print(f"{'='*60}")
    
    for name, (integral_func, metropolis_func) in implementations.items():
        print(f"\n{name}:")
        
        # Test intégrale
        start = time.time()
        pi_val = integral_func(num_trial)
        elapsed = time.time() - start
        benchmark_results[name]['integral'].append(elapsed)
        print(f"  Integral: π = {pi_val:.6f}, Time = {elapsed:.4f}s")
        
        # Test Metropolis
        start = time.time()
        pi_val = metropolis_func(num_trial)
        elapsed = time.time() - start
        benchmark_results[name]['metropolis'].append(elapsed)
        print(f"  Metropolis: π = {pi_val:.6f}, Time = {elapsed:.4f}s")

In [ ]:
# Visualisation comparative - Méthode Intégrale
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Graphique 1: Méthode Intégrale
for name in implementations.keys():
    ax1.plot(NUM_TRIALS, benchmark_results[name]['integral'], 'o-', label=name, linewidth=2, markersize=8)

ax1.set_xlabel('Number of trials', fontsize=12)
ax1.set_ylabel('Time (seconds)', fontsize=12)
ax1.set_title('Performance Comparison - Integral Method', fontsize=14, fontweight='bold')
ax1.set_xscale('log')
ax1.set_yscale('log')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Graphique 2: Méthode Metropolis
for name in implementations.keys():
    ax2.plot(NUM_TRIALS, benchmark_results[name]['metropolis'], 's-', label=name, linewidth=2, markersize=8)

ax2.set_xlabel('Number of trials', fontsize=12)
ax2.set_ylabel('Time (seconds)', fontsize=12)
ax2.set_title('Performance Comparison - Metropolis Method', fontsize=14, fontweight='bold')
ax2.set_xscale('log')
ax2.set_yscale('log')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Speedup par rapport à Python natif
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

baseline_integral = benchmark_results['Python Native']['integral']
baseline_metropolis = benchmark_results['Python Native']['metropolis']

# Speedup Intégrale
for name in implementations.keys():
    if name != 'Python Native':
        speedup = [baseline_integral[i] / benchmark_results[name]['integral'][i] 
                   for i in range(len(NUM_TRIALS))]
        ax1.plot(NUM_TRIALS, speedup, 'o-', label=name, linewidth=2, markersize=8)

ax1.axhline(y=1, color='red', linestyle='--', label='Baseline (Python Native)', linewidth=2)
ax1.set_xlabel('Number of trials', fontsize=12)
ax1.set_ylabel('Speedup (x times faster)', fontsize=12)
ax1.set_title('Speedup vs Python Native - Integral Method', fontsize=14, fontweight='bold')
ax1.set_xscale('log')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# Speedup Metropolis
for name in implementations.keys():
    if name != 'Python Native':
        speedup = [baseline_metropolis[i] / benchmark_results[name]['metropolis'][i] 
                   for i in range(len(NUM_TRIALS))]
        ax2.plot(NUM_TRIALS, speedup, 's-', label=name, linewidth=2, markersize=8)

ax2.axhline(y=1, color='red', linestyle='--', label='Baseline (Python Native)', linewidth=2)
ax2.set_xlabel('Number of trials', fontsize=12)
ax2.set_ylabel('Speedup (x times faster)', fontsize=12)
ax2.set_title('Speedup vs Python Native - Metropolis Method', fontsize=14, fontweight='bold')
ax2.set_xscale('log')
ax2.legend(fontsize=10)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Analyse des performances

### Résultats attendus et explications :

#### **1. Python Native**
- ❌ **Le plus lent** : boucles Python pures, interprétation à chaque itération
- Méthode intégrale légèrement plus rapide que Metropolis (pas de random, moins de conditions)

#### **2. Numpy**
- ✅ **10-50x plus rapide** que Python natif grâce à la vectorisation
- Les opérations sur arrays sont compilées en C
- Excellente performance pour Metropolis (génération de random vectorisée)

#### **3. Numba (JIT)**
- ✅ **50-100x plus rapide** que Python natif
- Compilation Just-In-Time du code Python en code machine
- Très efficace pour les boucles avec beaucoup d'itérations

#### **4. Numba Parallel**
- 🚀 **100-200x plus rapide** que Python natif (avec CPU multi-core)
- Parallélisation automatique avec `prange`
- Gain dépend du nombre de cœurs disponibles (2-4x sur la version JIT)

### Leçons importantes :

1. **Vectorisation** (Numpy) > Boucles Python : gain immédiat et facile
2. **JIT Compilation** (Numba) > Vectorisation : encore plus rapide pour boucles complexes
3. **Parallélisme** : bonus final une fois le code optimisé (2-4x supplémentaire)
4. La méthode **intégrale** est généralement plus rapide que **Metropolis** (moins aléatoire)

### Ordre de performance (du plus lent au plus rapide) :
1. 🐌 Python Native
2. 🏃 Generator/Lambda (similaire à Python native)
3. 🚴 Numpy (vectorisation)
4. 🏎️ Numba JIT (compilation)
5. 🚀 Numba Parallel (compilation + multi-threading)

## 3.4 Cython (Optionnel)

**Note** : Cython nécessite une compilation séparée. Voici un exemple de code :

```python
# %%cython
import numpy as np
cimport numpy as np
from libc.stdlib cimport rand, srand, RAND_MAX

def compute_pi_integral_cython(int num_trials):
    cdef double step = 1.0 / num_trials
    cdef double sum_val = 0.0
    cdef double x
    cdef int j
    
    for j in range(num_trials):
        x = (j - 0.5) * step
        sum_val += 4.0 / (1.0 + x * x)
    
    return sum_val * step
```

Pour utiliser Cython dans Jupyter :
```python
%load_ext Cython
```

Puis utiliser `%%cython` au début de la cellule.

## 3.6 Pandas et Dask (Optionnel)

**Note** : Pandas et Dask ne sont pas idéaux pour ce type de calcul (boucles itératives).
Ils excellent pour les opérations sur DataFrames (groupby, agrégations, etc.).

Exemple conceptuel avec Pandas :

```python
import pandas as pd

def compute_pi_pandas(num_trials):
    # Créer un DataFrame avec les indices
    df = pd.DataFrame({'j': range(num_trials)})
    step = 1.0 / num_trials
    
    # Calcul vectorisé
    df['x'] = (df['j'] - 0.5) * step
    df['value'] = 4.0 / (1.0 + df['x'] ** 2)
    
    return df['value'].sum() * step
```

Avec Dask (pour très grands datasets) :
```python
import dask.dataframe as dd

def compute_pi_dask(num_trials):
    # Version distribuée
    df = dd.from_pandas(pd.DataFrame({'j': range(num_trials)}), npartitions=4)
    step = 1.0 / num_trials
    
    df['x'] = (df['j'] - 0.5) * step
    df['value'] = 4.0 / (1.0 + df['x'] ** 2)
    
    return df['value'].sum().compute() * step
```

**Performance** : Plus lent que Numpy/Numba pour ce cas d'usage (overhead de DataFrame).

## Conclusion générale

### Résumé des performances pour le calcul de π :

| Méthode | Speedup (vs Python) | Complexité | Quand l'utiliser |
|---------|---------------------|------------|------------------|
| **Python Native** | 1x (baseline) | Simple | Prototypage rapide |
| **Generator/Lambda** | ~1x | Simple | Économie mémoire, mais pas plus rapide |
| **Numpy** | 10-50x | Moyenne | Opérations vectorisables |
| **Numba JIT** | 50-100x | Moyenne | Boucles complexes, calcul intensif |
| **Numba Parallel** | 100-200x | Moyenne | Multi-core, grandes données |
| **Cython** | 50-100x | Difficile | Performance critique, intégration C |
| **Pandas/Dask** | < Numpy | Difficile | Analyse de données, pas calcul numérique |

### Recommandations :

1. **Commencez simple** : Python natif pour le prototype
2. **Vectorisez** : Passez à Numpy si possible (gain facile)
3. **JIT Compilation** : Utilisez Numba pour les boucles qui ne se vectorisent pas bien
4. **Parallélisez** : Ajoutez `parallel=True` et `prange` pour gain supplémentaire
5. **Évitez** : Pandas/Dask pour du calcul numérique pur (overhead inutile)

### Pour ce TP :
- ✅ **Meilleure solution** : Numba Parallel (balance facilité/performance)
- ✅ **Alternative** : Numpy (si facilement vectorisable)
- ❌ **À éviter** : Python natif pour grandes données